Copy the file **spy_dte_0.csv.gz** located in https://github.com/crapher/medium/tree/main/23.OptionsStrategy/data to your colab folder

In [1]:
import warnings
import numpy as np
import pandas as pd

from tqdm import tqdm

In [2]:
# Constants
DEBUG = 1
OPTIONS_FILE='./spy_dte_0.csv.gz'

FEES_PER_CONTRACT = 0.6
CONTRACTS = 10

HOUR_OPEN = 10
MINUTE_OPEN = 30
STRIKES = 1
DIST_BETWEEN_STRIKES = 2
MAX_CHANGE_BEARISH = -0.35
MIN_CHANGE_BULLISH = 0.35

In [3]:
### Configuration ###
warnings.filterwarnings("ignore")
np.set_printoptions(suppress=True)
pd.options.mode.chained_assignment = None

In [4]:
### Helper functions ###
def get_vertical_option_kind(df, max_change_bearish, min_change_bullish):

    changes = 100 * (df['close_underlying'].values / df['close_underlying'].values[0] - 1)

    change = changes[-1]
    min_change = min(changes)
    max_change = max(changes)

    if change < max_change_bearish and max_change < min_change_bullish:
        return 'C' # Sell Call Vertical
    elif change > min_change_bullish and min_change > max_change_bearish:
        return 'P' # Sell Put Vertical
    else:
        return ''

def get_vertical_strikes(df, kind, strikes, dist_between_strikes):

    if kind == 'P':
        sell_strike = int(df['close_underlying'].values[0]) - strikes
        buy_strike  = sell_strike - dist_between_strikes
    else:
        sell_strike = int(df['close_underlying'].values[0]) + strikes
        buy_strike  = sell_strike + dist_between_strikes

    return (buy_strike, sell_strike)

def get_vertical_result(df, kind, buy_strike, sell_strike):

    # Open Vertical
    df = df.sort_values('date', ascending=True)

    stock_price = df['close_underlying'].values[0]
    buy_price   = df[(df['strike'] == buy_strike)  & (df['kind'] == kind)]['close'].values[0]
    sell_price  = df[(df['strike'] == sell_strike) & (df['kind'] == kind)]['close'].values[0]

    open_cost  = -100 * buy_price
    open_cost +=  100 * sell_price
    open_cost *= CONTRACTS

    if DEBUG:
        tqdm.write(f' OPEN {kind} ({stock_price:6.2f}): SELL {sell_strike} @ {sell_price:6.2f} -  BUY {buy_strike} @ {buy_price:6.2f} - TOTAL: {(-buy_price + sell_price):6.2f}')

    # Close Vertical (Prices are 0 if they are OTM/ATM or the difference between the strike and the underlying's close price if they are ITM)
    stock_price = df['close_underlying'].values[-1]
    sell_price = 0 if (kind == 'P' and buy_strike <= stock_price) or (kind == 'C' and buy_strike >= stock_price) else abs(buy_strike - stock_price)
    buy_price = 0 if (kind == 'P' and sell_strike <= stock_price) or (kind == 'C' and sell_strike >= stock_price) else abs(sell_strike - stock_price)

    if DEBUG:
        tqdm.write(f'CLOSE {kind} ({stock_price:6.2f}):  BUY {sell_strike} @ {buy_price:6.2f} - SELL {buy_strike} @ {sell_price:6.2f} - TOTAL: {(-buy_price + sell_price):6.2f}')

    close_cost = -100 * buy_price
    close_cost += 100 * sell_price
    close_cost *= CONTRACTS

    if DEBUG:
        tqdm.write(f'**************** Processed Date: {df["date"].dt.date.values[0]} - Result: {open_cost + close_cost:8.2f} ****************')

    return open_cost + close_cost

In [5]:
### Read File ###
df = pd.read_csv(OPTIONS_FILE, header=0)
df['date'] = pd.to_datetime(df['date'])

In [6]:
### Get all the expiration dates ###
expirations = df['expire_date'].unique().tolist()

In [7]:
### Get P&L by day ###
pnl = []
df_trend = df[(df.date.dt.hour < HOUR_OPEN) | ((df.date.dt.hour == HOUR_OPEN) & (df.date.dt.minute < MINUTE_OPEN))]
df_cost = df[(df.date.dt.hour > HOUR_OPEN) | ((df.date.dt.hour == HOUR_OPEN) & (df.date.dt.minute >= MINUTE_OPEN))]
df_open = df_cost[(df_cost.date.dt.hour == HOUR_OPEN) & (df_cost.date.dt.minute == MINUTE_OPEN)]

for expiration in tqdm(expirations):

    try:
        kind = get_vertical_option_kind(df_trend[df_trend['expire_date'] == expiration], MAX_CHANGE_BEARISH, MIN_CHANGE_BULLISH)

        if kind == '':
            if DEBUG:
                tqdm.write(f'**************** Processed Date: {expiration} - Result: -- NA -- ****************')

            continue

        (buy_strike, sell_strike) = get_vertical_strikes(df_open[df_open['expire_date'] == expiration], kind, STRIKES, DIST_BETWEEN_STRIKES)
        result = get_vertical_result(df_cost[df_cost['expire_date'] == expiration], kind, buy_strike, sell_strike)

        pnl.append([expiration, result])

    except Exception as ex:
        continue

  1%|          | 4/370 [00:00<00:39,  9.17it/s]

 OPEN P (419.24): SELL 418 @   0.38 -  BUY 416 @   0.12 - TOTAL:   0.26
CLOSE P (420.84):  BUY 418 @   0.00 - SELL 416 @   0.00 - TOTAL:   0.00
**************** Processed Date: 2021-06-21 - Result:   260.00 ****************
**************** Processed Date: 2021-06-23 - Result: -- NA -- ****************
**************** Processed Date: 2021-06-25 - Result: -- NA -- ****************
**************** Processed Date: 2021-06-28 - Result: -- NA -- ****************
**************** Processed Date: 2021-06-30 - Result: -- NA -- ****************


**************** Processed Date: 2021-07-02 - Result: -- NA -- ****************


  2%|▏         | 7/370 [00:00<00:38,  9.50it/s]

 OPEN C (432.06): SELL 433 @   0.18 -  BUY 435 @   0.02 - TOTAL:   0.16
CLOSE C (432.90):  BUY 433 @   0.00 - SELL 435 @   0.00 - TOTAL:   0.00
**************** Processed Date: 2021-07-06 - Result:   160.00 ****************
**************** Processed Date: 2021-07-07 - Result: -- NA -- ****************


...

**************** Processed Date: 2023-06-06 - Result: -- NA -- ****************
**************** Processed Date: 2023-06-07 - Result: -- NA -- ****************
**************** Processed Date: 2023-06-08 - Result: -- NA -- ****************


 99%|█████████▉| 366/370 [00:46<00:00,  9.66it/s]

 OPEN P (431.90): SELL 430 @   0.22 -  BUY 428 @   0.07 - TOTAL:   0.15
CLOSE P (429.89):  BUY 430 @   0.11 - SELL 428 @   0.00 - TOTAL:  -0.11
**************** Processed Date: 2023-06-09 - Result:    40.00 ****************
**************** Processed Date: 2023-06-12 - Result: -- NA -- ****************
**************** Processed Date: 2023-06-13 - Result: -- NA -- ****************
**************** Processed Date: 2023-06-14 - Result: -- NA -- ****************


100%|██████████| 370/370 [00:46<00:00,  7.99it/s]

 OPEN P (439.30): SELL 438 @   0.52 -  BUY 436 @   0.17 - TOTAL:   0.35
CLOSE P (442.60):  BUY 438 @   0.00 - SELL 436 @   0.00 - TOTAL:   0.00
**************** Processed Date: 2023-06-15 - Result:   350.00 ****************
**************** Processed Date: 2023-06-16 - Result: -- NA -- ****************


In [8]:
### Generate result ###

# Calculate Net Result
df_result = pd.DataFrame(pnl, columns=['expire_date','gross_result'])
df_result['net_result'] = df_result['gross_result'] - CONTRACTS * FEES_PER_CONTRACT * 2 # Let them expire
df_result['wins'] = np.where(df_result['net_result'] > 0, 1, 0)
df_result['losses'] = np.where(df_result['net_result'] < 0, 1, 0)

In [9]:
# Show the parameters
print(f' PARAMETERS '.center(70, '*'))
print(f'* Hour Open: {HOUR_OPEN}')
print(f'* Minute Open: {MINUTE_OPEN}')
print(f'* Strikes From Price: {STRIKES}')
print(f'* Distance Between Strikes: {DIST_BETWEEN_STRIKES}')
print(f'* Max % Change To Open A Bearish Trade: {MAX_CHANGE_BEARISH}')
print(f'* Min % Change To Open A Bullish Trade: {MIN_CHANGE_BULLISH}')

# Show the Total Result
print(f' SUMMARIZED RESULT '.center(70, '*'))
print(f'* Trading Days: {len(df_result)}')
print(f'* Gross PnL: $ {df_result["gross_result"].sum():.2f}')
print(f'* Net PnL: $ {df_result["net_result"].sum():.2f}')
print(f'* Win Rate: {100 * (df_result["wins"].sum() / (df_result["wins"].sum() + df_result["losses"].sum())):.2f} %')

# Show The Monthly Result
print(f' MONTHLY DETAIL RESULT '.center(70, '*'))
df_monthly = df_result[['expire_date','gross_result','net_result','wins','losses']]
df_monthly['year_month'] = df_monthly['expire_date'].str[0:7]
df_monthly = df_monthly.drop('expire_date', axis=1)
df_monthly = df_monthly.groupby(['year_month']).sum()

print(df_monthly)


***************************** PARAMETERS *****************************
* Hour Open: 10
* Minute Open: 30
* Strikes From Price: 1
* Distance Between Strikes: 2
* Max % Change To Open A Bearish Trade: -0.35
* Min % Change To Open A Bullish Trade: 0.35
************************* SUMMARIZED RESULT **************************
* Trading Days: 130
* Gross PnL: $ 32510.10
* Net PnL: $ 30950.10
* Win Rate: 80.77 %
*********************** MONTHLY DETAIL RESULT ************************
            gross_result  net_result  wins  losses
year_month                                        
2021-06            260.0       248.0     1       0
2021-07            780.0       732.0     3       1
2021-08            440.0       416.0     2       0
2021-09           2230.1      2170.1     5       0
2021-10          -3480.0     -3540.0     2       3
2021-11          -2710.0     -2734.0     0       2
2021-12           1980.0      1920.0     4       1
2022-01            180.0       144.0     2       1
2022-02     